<a href="https://colab.research.google.com/github/andrawaag/Gene_Wiki_Wikibase/blob/master/notebooks/DiseaseOntologyBot_on_Wikibase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Source: https://colab.research.google.com/drive/1jSn7vZ9W_AiIUn___KUVdG5YoVK58LWW

In [0]:
!pip install wikidataintegrator

     |████████████████████████████████| 71kB 2.1MB/s 
     |████████████████████████████████| 92kB 4.7MB/s 
     |████████████████████████████████| 51kB 6.3MB/s 
     |████████████████████████████████| 235kB 24.2MB/s 
     |████████████████████████████████| 112kB 33.8MB/s 
     |████████████████████████████████| 71kB 8.7MB/s 
     |████████████████████████████████| 61kB 7.5MB/s 
     |████████████████████████████████| 61kB 8.1MB/s 
     |████████████████████████████████| 51kB 7.2MB/s 
     |████████████████████████████████| 112kB 31.4MB/s 
  Created wheel for wikidataintegrator: filename=wikidataintegrator-0.6.0-cp36-none-any.whl size=73229 sha256=c90ca87db70b870db1b8290bb34ccdaa1f07f41171d34ab7ba7efb68e24e8390
  Stored in directory: /root/.cache/pip/wheels/40/94/f8/8815e5f953d2b19db82c278303fc8cf7395116b5cb5cc30872
  Created wheel for simplejson: filename=simplejson-3.17.0-cp36-cp36m-linux_x86_64.whl size=114199 sha256=52ad060bffa8fc3ad6cf35abb71b286dc0ee733f04a0abdf6686c1161075bee1
 

In [0]:
from wikidataintegrator import wdi_core, wdi_login
import os
from rdflib import Graph, URIRef
import pandas as pd

# This code is to Login to Wikibase. The pattern of api is the URL of the wikibase + /w/api.php
# In this example that is https://diseases.semscape.org/w/api.php


## The following section is to download the orginal Disease ontology
print("\nDownloading the Disease Ontology...")
url = "https://raw.githubusercontent.com/DiseaseOntology/HumanDiseaseOntology/master/src/ontology/releases/2020-04-20/doid.owl"

doGraph = Graph()
doGraph.parse(url, format="xml")

<Graph identifier=Nb6e74209632c4c2881cbd72dc3934ba5 (<class 'rdflib.graph.Graph'>)>

In [0]:
import pprint
df_doNative = pd.DataFrame(columns=["do_uri", "doid", "label", "subclassof", "aliases"])

qres = doGraph.query(
    """
       PREFIX obo: <http://www.geneontology.org/formats/oboInOwl#>
       PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
       PREFIX oboInOwl: <http://www.geneontology.org/formats/oboInOwl#>

       SELECT DISTINCT ?do_uri ?doid ?label (GROUP_CONCAT(?subClassOf;separator="|") as ?subclasses) (GROUP_CONCAT(?exactsynonym;separator="|") as ?exact_synonyms)
       WHERE {
          ?do_uri obo:id ?doid ;
                  rdfs:label ?label .
         OPTIONAL {?do_uri rdfs:subClassOf ?subClassOf ;}
         OPTIONAL {?do_uri oboInOwl:hasExactSynonym ?exactsynonym}
       }
       GROUP BY ?do_uri """)

for row in qres:
    df_doNative = df_doNative.append({
     "do_uri": str(row[0]),
     "doid": str(row[1]),
     "label":  str(row[2]),
     "subclassof": str(row[3]),
     "aliases": str(row[4])
      }, ignore_index=True)

df_doNative


,do_uri,doid,label,subclassof,aliases
0,http://purl.obolibrary.org/obo/DOID_11186,DOID:11186,allescheriosis,Nc199b925fc56418aa1cf94e85702eb70|N4a197a3f80d...,Petriellidosis|Petriellidosis|Petriellidosis
1,http://purl.obolibrary.org/obo/DOID_4766,DOID:4766,embryoma,http://purl.obolibrary.org/obo/DOID_688,Blastoma
2,http://purl.obolibrary.org/obo/DOID_6083,DOID:6083,childhood ovarian endodermal sinus tumor,http://purl.obolibrary.org/obo/DOID_5350|http:...,childhood ovarian yolk sac tumor|childhood ova...
3,http://purl.obolibrary.org/obo/DOID_11452,DOID:11452,perinatal jaundice due to hepatocellular damage,http://purl.obolibrary.org/obo/DOID_2383,Perinatal jaundice due to hepatocellular damage
4,http://purl.obolibrary.org/obo/DOID_6271,DOID:6271,gastric cardia adenocarcinoma,N5887ec61a27848458e003827c2a79ccf|http://purl....,adenocarcinoma of Cardia of stomach|adenocarci...
...,...,...,...,...,...
12487,http://purl.obolibrary.org/obo/DOID_7078,DOID:7078,obsolete malignant mediastinal peripheral nerv...,,MPNST of mediastinum
12488,http://purl.obolibrary.org/obo/DOID_0110673,DOID:0110673,congenital myasthenic syndrome 19,http://purl.obolibrary.org/obo/DOID_0050737|Ne...,CMS19|CMS19|CMS19
12489,http://purl.obolibrary.org/obo/DOID_0080333,DOID:0080333,aortic valve disease 1,http://purl.obolibrary.org/obo/DOID_0080332,
12490,http://purl.obolibrary.org/obo/DOID_5158,DOID:5158,pleural cancer,http://purl.obolibrary.org/obo/DOID_201|http:/...,neoplasm of pleura|Pleural tumor


In [0]:
from wikidataintegrator.wdi_helpers import try_write

wikibase = "https://diseases.semscape.org/w/api.php"
WBUSER = getpass(prompt="username:")  
WBPASS = getpass(prompt='Enter your password: ')  
login = wdi_login.WDLogin(WBUSER, WBPASS, mediawiki_api_url=wikibase)

query = """
  PREFIX wbt: <http://diseases.semscape.org/prop/direct/>
  SELECT * WHERE {?item wbt:P4 ?doid}
"""
existing_do = dict()
results =  wdi_core.WDItemEngine.execute_sparql_query(query=query, endpoint="https://diseases.semscape.org/query/sparql")

for result in results["results"]["bindings"]:
    existing_do[result["doid"]["value"]] = result["item"]["value"]

for index, row in df_doNative.iterrows():
  data = []
  #disease ontology ID
  data.append(wdi_core.WDExternalID(row["doid"], prop_nr="P4"))

  if row["doid"] in existing_do.keys():
    continue
    #qid = existing_do[row["doid"]].replace("http://diseases.semscape.org/entity/", "")
    #print(qid)
    #wb_do_item = wdi_core.WDItemEngine(wd_item_id=qid, data=data, mediawiki_api_url=wikibase, sparql_endpoint_url="https://diseases.semscape.org/query/sparql")
  else:
    wb_do_item = wdi_core.WDItemEngine(data=data, mediawiki_api_url=wikibase, sparql_endpoint_url="https://diseases.semscape.org/query/sparql")
                      
  wb_do_item.set_label(row["label"], lang="en")
  wb_do_item.set_description("human disease", lang="en")
  

  if row["aliases"] != "":
    wb_do_item.set_aliases(row["aliases"].split("|"), lang="en")
  pprint.pprint(wb_do_item.get_wd_json_representation())
  try_write(wb_do_item, record_id=row["doid"], record_prop="P4", edit_summary="Updated a Disease Ontology", login=login)


https://diseases.semscape.org/w/api.php
Successfully logged in as Andrawaag


/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py:219: UserWarning: Warning: No distinct value properties found
Please set P2302 and Q21502410 in your wikibase or set `core_props` manually.
Continuing with no core_props
  "Continuing with no core_props")
/usr/local/lib/python3.6/dist-packages/wikidataintegrator/wdi_core.py:175: UserWarning: mapping relation types are being ignored
  warnings.warn("mapping relation types are being ignored")


{'aliases': {'en': [{'language': 'en', 'value': 'Papillary syringadenoma'},
                    {'language': 'en',
                     'value': 'Syringocystadenoma papilliferum'}]},
 'claims': {'P4': [{'mainsnak': {'datatype': 'external-id',
                                 'datavalue': {'type': 'string',
                                               'value': 'DOID:5445'},
                                 'property': 'P4',
                                 'snaktype': 'value'},
                    'qualifiers': {},
                    'qualifiers-order': [],
                    'rank': 'normal',
                    'references': [],
                    'type': 'statement'}]},
 'descriptions': {'en': {'language': 'en', 'value': 'human disease'}},
 'labels': {'en': {'language': 'en',
                   'value': 'syringocystadenoma papilliferum'}}}
Error while writing to Wikidata
{'error': {'code': 'modification-failed', 'info': 'Item [[Item:Q8341|Q8341]] already has label "syringocystad

In [0]:
query = """
  PREFIX wbt: <http://diseases.semscape.org/prop/direct/>
  SELECT * WHERE {?item wbt:P4 ?doid}
"""
existing_do = dict()
results =  wdi_core.WDItemEngine.execute_sparql_query(query=query, endpoint="https://diseases.semscape.org/query/sparql")
pprint.pprint(results)
for result in results["results"]["bindings"]:
    existing_do[result["doid"]["value"]] = result["item"]["value"]

existing_do

{'head': {'vars': ['item', 'doid']},
 'results': {'bindings': [{'doid': {'type': 'literal', 'value': 'DOID:0080600'},
                           'item': {'type': 'uri',
                                    'value': 'http://diseases.semscape.org/entity/Q1'}},
                          {'doid': {'type': 'literal', 'value': 'DOID:2841'},
                           'item': {'type': 'uri',
                                    'value': 'http://diseases.semscape.org/entity/Q3'}},
                          {'doid': {'type': 'literal', 'value': 'DOID:12311'},
                           'item': {'type': 'uri',
                                    'value': 'http://diseases.semscape.org/entity/Q4'}},
                          {'doid': {'type': 'literal', 'value': 'DOID:0050451'},
                           'item': {'type': 'uri',
                                    'value': 'http://diseases.semscape.org/entity/Q8'}},
                          {'doid': {'type': 'literal', 'value': 'DOID:0110056'},
   

{'DOID:0050451': 'http://diseases.semscape.org/entity/Q8',
 'DOID:0070258': 'http://diseases.semscape.org/entity/Q9',
 'DOID:0080600': 'http://diseases.semscape.org/entity/Q1',
 'DOID:0110056': 'http://diseases.semscape.org/entity/Q5',
 'DOID:0111545': 'http://diseases.semscape.org/entity/Q6',
 'DOID:12311': 'http://diseases.semscape.org/entity/Q4',
 'DOID:2841': 'http://diseases.semscape.org/entity/Q3',
 'DOID:9369': 'http://diseases.semscape.org/entity/Q7'}

In [0]:
qres = doGraph.query(
    """
      

       SELECT ?source ?target WHERE {

       ?s <http://www.w3.org/2004/02/skos/core#exactMatch> ?o ;
          <http://www.w3.org/2002/07/owl#annotatedSource> ?source ;
          <http://www.w3.org/2002/07/owl#annotatedTarget> ?target .


  }
       """)

for row in qres:
  print(row[0], row[1])

http://purl.obolibrary.org/obo/DOID_11870 MESH:D020774
http://purl.obolibrary.org/obo/DOID_2377 MESH:D009103
